#Assignment: Spark SQL and Data Frames

Dataset: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page (February 2021)

Tech Stack:

1.   PySpark
2.   Google BigQuery



In [1]:
import findspark
findspark.init()
findspark.find()

'/opt/spark'

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("archiecm").config('spark.ui.port', '4050').getOrCreate()

23/03/18 10:47:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

In [4]:
df = spark.read.parquet("fhvhv_tripdata_2021-02.parquet", header=True, inferSchema=True)
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [5]:
rows = df.count()
cols = len(df.columns)

print(f'Dimensions of Data: {(rows,cols)}')
print(f'Rows of Data: {rows}')
print(f'Columns of Data: {cols}')

Dimensions of Data: (11613942, 24)
Rows of Data: 11613942
Columns of Data: 24


In [6]:
df.show(5)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

#### How many Taxi Trips were there on February 15?

In [7]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *

In [8]:
df = df \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

df.registerTempTable('data_table')

In [9]:
total_taxi_trip_0215 = spark.sql(""" 

    SELECT COUNT(pickup_datetime) AS total_taxi_trip_0215
    FROM data_table
    WHERE pickup_datetime >= '2021-02-15 00:00:00' AND pickup_datetime < '2021-02-16 00:00:00'

""")

total_taxi_trip_0215.show()

+--------------------+
|total_taxi_trip_0215|
+--------------------+
|              367170|
+--------------------+



#### Find the longest trip for each day.

In [10]:
df.createOrReplaceTempView('data_view')

In [11]:
longesttrip_eachday = df.withColumn("pickup_datetime" , to_date(df['pickup_datetime']))\
                      .select(['pickup_datetime','trip_miles'])\
                      .where("pickup_datetime >= '2021-02-01' ")\
                      .groupby(F.col('pickup_datetime')).agg(F.max('trip_miles').alias('longest_trip')).sort(desc("longest_trip"))
longesttrip_eachday.show(10)

+---------------+------------+
|pickup_datetime|longest_trip|
+---------------+------------+
|     2021-02-18|      527.11|
|     2021-02-10|       512.5|
|     2021-02-09|      480.73|
|     2021-02-27|      454.49|
|     2021-02-22|      347.41|
|     2021-02-20|      340.64|
|     2021-02-19|      329.16|
|     2021-02-17|      324.19|
|     2021-02-16|     307.661|
|     2021-02-24|      301.73|
+---------------+------------+
only showing top 10 rows



#### Find top 5 most frequent 'dispatching_base_num'.

In [12]:
top5_frequent_dbm = df.groupBy("dispatching_base_num").count() \
                    .orderBy(F.col('count').desc())

top5_frequent_dbm.show(5)

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
+--------------------+-------+
only showing top 5 rows



#### Find top 5 most common location pairs (PULocationID and DOLocationID).

In [13]:
top5_location_pairs = df.where("PUlocationID IS NOT NULL AND DOlocationID IS NOT NULL") \
                      .groupBy(["PUlocationID",'DOlocationID']) \
                      .count() \
                      .orderBy(F.col('count').desc())
top5_location_pairs.show(5)

+------------+------------+-----+
|PUlocationID|DOlocationID|count|
+------------+------------+-----+
|          76|          76|45041|
|          26|          26|37329|
|          39|          39|28026|
|          61|          61|25976|
|          14|          14|17934|
+------------+------------+-----+
only showing top 5 rows



#### Write all of the result to BigQuery table.

In [14]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ".google/credentials/google_credentials.json"

In [44]:
# https://googleapis.dev/python/pandas-gbq/latest/writing.html
import pandas_gbq
import pandas as pd

# TODO: Set project_id to your Google Cloud Platform project ID.
project_id = "data-fellowship-9-project"

# TODO: Set dataset_id to the full destination dataset ID.
dataset_id = 'taxi_trip_spark'

In [45]:
df_total_taxi_trip_0215 = total_taxi_trip_0215.toPandas()
df_longesttrip_eachday = longesttrip_eachday.toPandas()
df_top5_frequent_dbm = top5_frequent_dbm.toPandas()
df_top5_location_pairs = top5_location_pairs.toPandas()

In [48]:
df_longesttrip_eachday['pickup_datetime'] = df_longesttrip_eachday['pickup_datetime'].astype(str)
df_longesttrip_eachday['pickup_datetime'] = df_longesttrip_eachday['pickup_datetime'].apply(pd.to_datetime)

In [51]:
def load_to_bq(df, table_name):
    pandas_gbq.to_gbq(df, f'{dataset_id}.{table_name}', project_id=project_id)

load_to_bq(df_total_taxi_trip_0215, 'taxi_trip_0215')
load_to_bq(df_longesttrip_eachday, 'longesttrip_eachday')
load_to_bq(df_top5_frequent_dbm, 'top5_frequent_dbm')
load_to_bq(df_top5_location_pairs, 'top5_location_pairs')

100%|██████████████| 1/1 [00:00<00:00, 20262.34it/s]
